In [7]:
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.callbacks import Callback
from keras.layers import BatchNormalization, LayerNormalization

In [8]:
import import_ipynb
from dataset_config import create_first_data_flow

importing Jupyter notebook from dataset_config.ipynb
Found 0 images belonging to 13 classes.
Found 0 images belonging to 0 classes.


In [1]:
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet import ResNet101

In [3]:
# incomplete because we need to generate a better dataset for training
train_data = 1
test_data = 2

In [4]:
class TestCallback(Callback):
  def __init__(self, train_data, test_data, verbose=False):
    self.train_data = train_data
    self.test_data = test_data

    self.train_log = {}
    self.test_log = {}
    self.verbose = verbose

  def on_epoch_end(self, epoch, logs={}):
    x_train, y_train = self.train_data
    x_test, y_test = self.test_data

    train_loss, train_acc = self.model.evaluate(x_train, y_train, batch_size=32, verbose=0)
    test_loss, test_acc = self.model.evaluate(x_test, y_test, batch_size=32,  verbose=0)

    self.train_log[epoch] = train_acc
    self.test_log[epoch] = test_acc

    if self.verbose == True:
      log = 'E: {} - Train - Test loss: {} - {}, acc: {} - {}'.format(
              epoch, train_loss, test_loss, train_acc, test_acc)
      print(log)

In [5]:
def init_resnet_arch(resnet_model):
    model = Sequential()
    model.add(resnet_model)
    model.add(Flatten())
    model.add(Dense(1024, activation=('relu')))
    model.add(BatchNormalization())
    # model.add(Dropout(0.4))
    model.add(Dense(512, activation=('relu')))
    model.add(BatchNormalization())
    # model.add(Dropout(0.4))
    model.add(Dense(256, activation=('relu')))
    model.add(BatchNormalization())
    # model.add(Dropout(0.4))
    model.add(Dense(128, activation=('relu')))
    model.add(BatchNormalization())
    # model.add(Dropout(0.4))
    model.add(Dense(10, activation=('softmax')))
    return model

In [6]:
def resnet_model_compile():
    model_resnet = ResNet50(include_top=False, weights='imagenet', input_shape=(800,800,3), classes=train_data.shape[1])
    model_resnet_sgd = init_resnet_arch(model_resnet)

    # 391 steps per epoch-- 7 * 391 = 2737 steps
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate = 1e-2,
        decay_steps = 2730,
        decay_rate = 0.7)

    # 1e-2 the default value for SGD and 1e-3 default value for Adam
    optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)

    model_resnet_sgd.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    baseline_callbacks = [TestCallback(train_data, test_data)]
    fit_res = model_resnet.fit(train_data, test_data, epochs=30, callbacks=baseline_callbacks, batch_size=128, verbose=False)
    return fit_res